## Importing libraries

In [1]:
!pip install torch torchmetrics torchaudio datasets transformers scikit-learn matplotlib wandb torchcodec

from datasets import load_dataset
from transformers import (
    AutoFeatureExtractor,
    AutoModelForAudioClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
import librosa
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
)
from google.colab import drive
import os
from google.colab import userdata
import torch
import wandb
import torchmetrics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.6 MB/s eta 0:00:00


## Getting paths for metadata and getting a key for wandb

In [16]:
BASE_DIR = '/content/'

TRAIN_PATH = os.path.join(BASE_DIR, 'train.csv')
VALID_PATH = os.path.join(BASE_DIR, 'valid.csv')
TEST_PATH  = os.path.join(BASE_DIR, 'test.csv')

wandb_kay = userdata.get('WANDB')

## Log in to wandb

In [17]:
wandb.login(key=wandb_kay)

wandb.init(
    project="AST",
)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: boklahbohdan (detect_kaggle) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Cloning your google drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


## Selecting the target directory

In [4]:
!rsync -a "/content/drive/MyDrive/audio_cls/" .

## Downloading a dataset

In [5]:
dataset = load_dataset(
    "csv",
    data_files={
        "train": TRAIN_PATH,
        "valid": VALID_PATH,
        "test" : TEST_PATH
    }
)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['path', 'target'],
        num_rows: 480
    })
    valid: Dataset({
        features: ['path', 'target'],
        num_rows: 96
    })
    test: Dataset({
        features: ['path', 'target'],
        num_rows: 105
    })
})

## fix of path for a google colab

In [7]:
DATA_DIR_TRAIN = "/content/train"
DATA_DIR_VALID = "/content/valid"
DATA_DIR_TEST  = "/content/test"

def fix_path_train(example):
    example["path"] = f"{DATA_DIR_TRAIN}/{example["path"]}"
    return example

def fix_path_valid(example):
    example["path"] = f"{DATA_DIR_VALID}/{example["path"]}"
    return example

def fix_path_test(example):
    example["path"] = f"{DATA_DIR_TEST}/{example["path"]}"
    return example

def fix_label(example):
    example["labels"] = example.pop("target")
    return example

train_df = dataset['train']
valid_df = dataset['valid']
test_df  = dataset['test']

train_df = train_df.map(fix_path_train)
valid_df = valid_df.map(fix_path_valid)
test_df  = test_df.map(fix_path_test)

train_df = train_df.map(fix_label)
valid_df = valid_df.map(fix_label)
test_df  = test_df.map(fix_label)


Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

## Initial model

In [8]:
MODEL_NAME = "MIT/ast-finetuned-audioset-10-10-0.4593"
NUM_LABELS = 4

feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_NAME)

model = AutoModelForAudioClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    ignore_mismatched_sizes=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ASTForAudioClassification were not initialized from the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- classifier.dense.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.dense.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Downloading audio samples

In [10]:
def preprocess(batch):
    waveform, sr = librosa.load(batch["path"], sr=16000)
    inputs = feature_extractor(
        waveform,
        sampling_rate=16000,
        return_tensors="pt"
    )
    batch["input_values"] = inputs["input_values"].squeeze(0).tolist()
    return batch

train_df = train_df.map(preprocess)
valid_df = valid_df.map(preprocess)
test_df  = test_df.map(preprocess)

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

## Implement metrics computing

In [12]:
def compute_metrics_hf(eval_pred):
    logits, labels = eval_pred
    logits = torch.tensor(logits)
    labels = torch.tensor(labels).long()
    probs = torch.nn.functional.softmax(logits, dim=-1)
    preds = probs.argmax(dim=-1)
    labels_np = labels.cpu().numpy()
    preds_np = preds.cpu().numpy()
    return {
        "accuracy": accuracy_score(labels_np, preds_np),
        "f1_macro": f1_score(labels_np, preds_np, average="macro"),
        "precision_macro": precision_score(labels_np, preds_np, average="macro"),
        "recall_macro": recall_score(labels_np, preds_np, average="macro"),
    }

## Implement a data collator

In [13]:
def collate_fn(features):
    xs = torch.stack(
        [torch.tensor(f["input_values"], dtype=torch.float32) for f in features]
    )
    labels = torch.tensor([f["labels"] for f in features], dtype=torch.long)
    return {"input_values": xs, "labels": labels}

## Initial a training config

In [21]:

train_batch_size = 48
val_batch_size = 24
EPOCHS = 200


early_stopping = EarlyStoppingCallback(
    early_stopping_patience=5,
    early_stopping_threshold=0.0
)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=val_batch_size,
    num_train_epochs=EPOCHS,
    eval_strategy="steps",
    save_steps=10,
    eval_steps=10,
    logging_strategy="steps",
    logging_steps=10,
    report_to="wandb",
    fp16=False,
    gradient_checkpointing=True,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_df,
    eval_dataset=valid_df,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics_hf,
    data_collator=collate_fn,
    callbacks=[early_stopping]
)

/tmp/ipython-input-512696339.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Train a model

In [22]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision Macro,Recall Macro
10,0.044800,0.049863,0.989583,0.991831,0.992857,0.991071
20,0.016600,0.001281,1.000000,1.000000,1.000000,1.000000
30,0.000800,0.000290,1.000000,1.000000,1.000000,1.000000
40,0.000200,0.000071,1.000000,1.000000,1.000000,1.000000
50,0.000100,0.000048,1.000000,1.000000,1.000000,1.000000
60,0.000000,0.000033,1.000000,1.000000,1.000000,1.000000
70,0.000000,0.000026,1.000000,1.000000,1.000000,1.000000


TrainOutput(global_step=70, training_loss=0.008961173842960437, metrics={'train_runtime': 1610.2353, 'train_samples_per_second': 59.619, 'train_steps_per_second': 1.242, 'total_flos': 2.2775440607281152e+17, 'train_loss': 0.008961173842960437, 'epoch': 7.0})

In [23]:
trainer.save_model("./results/best_model")
feature_extractor.save_pretrained("./results/best_model")
artifact = wandb.Artifact("best_model", type="model")
artifact.add_dir("./results/best_model")
wandb.log_artifact(artifact)

wandb: Adding directory to artifact (results/best_model)... Done. 2.1s


<Artifact best_model>

In [24]:
predictions_output = trainer.predict(test_df)
print("Метрики на тесті:")
for k, v in predictions_output.metrics.items():
    print(f"{k}: {v:.4f}")
logits = predictions_output.predictions
probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)
preds = probs.argmax(dim=-1).numpy()
labels = predictions_output.label_ids

Метрики на тесті:
test_loss: 0.0736
test_accuracy: 0.9714
test_f1_macro: 0.9671
test_precision_macro: 0.9660
test_recall_macro: 0.9709
test_runtime: 21.7456
test_samples_per_second: 4.8290
test_steps_per_second: 0.2300
